# Sequence tagging using Markov Models and Baum-Welch probability



In [1]:
def transition_prob(a, b):
    """Equiprobable between all tokens, S is special case there is no transition to S"""
    return 1/3

def calculate_tag_sequences(phrase, Vocab):
    """
    Get's a phrase and a vocabulary and calculates all possible hidden states
    and their probabilities
    Example output:
    [[('S', 1.0), ('O', 0.33), ('N', 0.5), ('V', 0.5), ('O', 0.33), ('N', 0.5)], 
    [('S', 1.0), ('O', 0.33), ('N', 0.5), ('V', 0.5), ('O', 0.33), ('V', 0.5)], 
    [('S', 1.0), ('O', 0.33), ('O', 0.33), ('V', 0.5), ('O', 0.33), ('N', 0.5)], 
    [('S', 1.0), ('O', 0.33), ('O', 0.33), ('V', 0.5), ('O', 0.33), ('V', 0.5)]]
    
    """
    output = [[]]
    tokens = phrase.split()
    for t in tokens:
        token_tags = []
        for v in Vocab:
            if t in Vocab[v]:
                p_w_given_t = 1/len(Vocab[v])
                token_tags.append((v, p_w_given_t))
        

        tmp_output = []
        for i in range(len(output)):
            for tt in range(len(token_tags)):
                tmp_output.append( output[i]+ [token_tags[tt]])
        
        output = tmp_output
    
    return output

def calculate_tag_sequence_prob(sequence):
    prob = 1.0
    prev_step = ""
    for step in range(1, len(sequence)):
        prob *= transition_prob(sequence[step-1][0], sequence[step][0]) * sequence[step][1]

    return prob

def calculate_sequence_name(s):
    """
        Takes as input a list like:
        [("A", 0.3),("B", 0.2),("C", 0.5)]
        returns
        "ABC"
    """
    letters = map((lambda x: x[0]), s)
    combined = reduce((lambda x, y: x + y), letters )
    return combined

def calculate_baum_welch_prob(prob_dict, ab):
    """
    Calculate the Baum-Welch probability of a state given its previous state.
    Passed in are the calculated markov model probabilities
    e.g. P(V|O) is expressed as "OV"

    Input:
    prob_dict e.g. {'SONVON': 0.3, 'SONVOV': 0.3, 'SOOVON': 0.2, 'SOOVOV': 0.2}
    ab e.g. "OV"

    See https://www.coursera.org/learn/language-processing/supplement/CROHj/probabilities-of-tag-sequences-in-hmms
    """
    prob_numerator = 0
    prob_denominator = 0

    for name in prob_dict:
        count1 = 0
        count2 = 0
        for i in range(len(name)-1):
            token = name[i:i+2]
            # print(token)
            if ab == token:
                count1 += 1
            if token.startswith(ab[0:1]):
                count2 += 1

        prob_numerator += count1 * prob_dict[name]
        prob_denominator += count2 * prob_dict[name]
        assert(prob_denominator > 0)
    return prob_numerator/prob_denominator

In [2]:
from functools import reduce

Vocab = {
    "N": ["mimsy", "borogoves"],
    "V": ["were", "borogoves"],
    "O": ["all", "mimsy", "the"],
    "S": ["<s>"]
}

Phrase = "<s> all mimsy were the borogoves"

In [3]:
sequences = calculate_tag_sequences(Phrase, Vocab)
print(sequences)


[[('S', 1.0), ('O', 0.3333333333333333), ('N', 0.5), ('V', 0.5), ('O', 0.3333333333333333), ('N', 0.5)], [('S', 1.0), ('O', 0.3333333333333333), ('N', 0.5), ('V', 0.5), ('O', 0.3333333333333333), ('V', 0.5)], [('S', 1.0), ('O', 0.3333333333333333), ('O', 0.3333333333333333), ('V', 0.5), ('O', 0.3333333333333333), ('N', 0.5)], [('S', 1.0), ('O', 0.3333333333333333), ('O', 0.3333333333333333), ('V', 0.5), ('O', 0.3333333333333333), ('V', 0.5)]]


In [4]:
sequences_prob = [ calculate_tag_sequence_prob(seq)  for seq in sequences]
print(sequences_prob)


[5.715592135345221e-05, 5.715592135345221e-05, 3.810394756896814e-05, 3.810394756896814e-05]


In [5]:
norm_sequences_prob = { calculate_sequence_name(seq[0]) : seq[1] / sum(sequences_prob) for seq in zip(sequences, sequences_prob)}
print(norm_sequences_prob)


{'SONVON': 0.3, 'SONVOV': 0.3, 'SOOVON': 0.2, 'SOOVOV': 0.2}


## Baum Welch probability

In [6]:

calculate_baum_welch_prob(norm_sequences_prob, "OV")

0.37499999999999994